# Analyzing Fidenza Artblocks

Important: This notebook uses [Plotly](https://plotly.com/python/) mostly to visualize data. Plotly charts won't be rendered in Githubs notebook viewer. So please [view this notebook on nbviewer](https://nbviewer.org/github/ymyke/abalyzer/blob/main/fidenzas.ipynb) (or on your local installation after cloning the repo). Note that nbviewer caches notebooks and sometimes takes a long time to pick up a new version.

On the upside, all the Plotly charts are interactive. You can zoom, hover, pan, etc. Zooming is especially important due to the outliers in some the data so you can zoom in on the interesting parts.

In [37]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Get the Fidenzas

In [38]:
import datetime
from opensea import ARTBLOCK_CONTRACT, retrieve_assets

FIDENZA_IDS = list(range(78000000, 78000998 + 1))
assets = retrieve_assets(token_ids=FIDENZA_IDS, contract=ARTBLOCK_CONTRACT)
print(datetime.datetime.now())


.................................................. -- All 999 assets retrieved.
2021-10-12 14:53:12.313694


## Build a dataframe

In [39]:
import numpy as np
import pandas as pd
from helpers import turn_assets_into_df
df, traits = turn_assets_into_df(assets)

In [40]:
assert set(df.LPsymbol.unique()) == set(["ETH", "WETH", np.nan]), """
This sheet does not do currency conversion at the moment and therefore 
assumes all prices are in (W)ETH. But there are more symbols in the input 
data which would lead to apples being compared to ranges below. Aborting.
"""

## How many pieces have a last price on OpenSea?

In [41]:
print(f"{df[~df.Lastprice.isna()].shape[0] / df.shape[0]:.0%}")

61%


## How are prices evolving over time?

Note that this only takes into account the last sale of each piece.

In [42]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 8)
pd.plotting.register_matplotlib_converters()

fig = px.scatter(df, x="LPdate", y="Lastprice", hover_data=["Name", "Colors", "Probscore"])
fig.show()

## Price levels today

In [43]:
fig = px.histogram(df, x="Lastprice")
fig.show()

## How frequent are the different traits?

In [44]:
from IPython.display import display
for trait in traits:
    display(
        pd.DataFrame(df[trait].value_counts(normalize=False, sort=True, ascending=True)).transpose()
    )


,Relaxed,Anything Goes,No Overlap
Collision Check,17,30,952


,Dark Lifestyle,Party Time,White on Cream,Luxe-Derived,Cool,Rose,Black,AM,White Mono,Baked,Politique,Rad,Golf Socks,Luxe
Colors,6,9,10,11,12,22,26,30,37,47,64,86,111,528


,Low,High AF,Medium,High
Density,49,68,277,605


,No,Yes
Have Margin,427,572


,Yes,No
Outlined,100,899


...

,Sharp,Curved
Shape Angles,34,965


,Yes,No
Soft Shapes,150,849


,Yes,No
Spiral,35,964


,Yes,No
Super Blocks,241,758


,None,High,Low,Med
Turbulence,149,203,220,427


## How much do people pay for the different traits?

In [45]:
for trait in traits:

    fig = go.Figure()
    for traitvariant in (
        df.groupby(trait).median().sort_values("Lastprice", ascending=False).index
    ):
        fig.add_trace(
            go.Box(
                y=df[df[trait] == traitvariant].Lastprice.values,
                name=traitvariant,
                boxpoints="all",
                jitter=0.2,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            )
        )
    fig.update_layout(title=trait)
    fig.show()


**Some Observations**:

- The traits with the biggest price impact seem to be Spiral, Shape Angles, Scale, Colors, Collision Check.
- All other traits don't make much of a difference in terms of price.

## Probscore as a simple unified metric to predict prices

The Probscore metric is simply the product of the probabilities of the different traits of a piece. There is currently not much value in that metric. While only the lowest probabilities are able to command the very high outlier prices, overall correlation between Probscore and Lastprice remains low:

In [46]:
fig = px.scatter(df, x="Probscore", y="Lastprice")
fig.show()
df[["Probscore", "Lastprice"]].corr()

,Probscore,Lastprice
Probscore,1.000000,-0.032479
Lastprice,-0.032479,1.000000
